## Pandas DataFrame статистика и ускорение

## 1. Простые статистические операции

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sps

In [ ]:
df = pd.DataFrame(sps.norm.rvs(size=(10, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,0.356587,0.677383,-0.808678,0.436242
1,0.118757,1.001112,0.271759,0.409493
2,0.341055,-0.810651,0.393015,-0.754005
3,-1.186959,0.289895,0.884809,2.085891
4,0.272135,1.395138,0.687555,0.075547
5,1.806818,-0.458174,2.906355,-0.256508
6,0.618698,-0.470176,-0.559366,0.329487
7,1.677485,2.029751,0.783604,-0.513796
8,-0.821490,0.948548,0.817842,0.378035
9,-2.354276,-0.497804,-0.779210,0.383991


Выведем описательные статистики по столбцам &mdash; количество значений, среднее, стандартное отклонение (корень из дисперсии), минимум, квантили, максимум.

In [ ]:
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.082881,0.410502,0.459769,0.257438
std,1.262740,0.952306,1.089030,0.770939
min,-2.354276,-0.810651,-0.808678,-0.754005
25%,-0.586428,-0.467176,-0.351584,-0.173494
50%,0.306595,0.483639,0.540285,0.353761
75%,0.553170,0.987971,0.809283,0.403118
max,1.806818,2.029751,2.906355,2.085891


Среднее по столбцам

In [ ]:
df.mean()

,0
A,0.082881
B,0.410502
C,0.459769
D,0.257438


In [ ]:
df.corr()

,A,B,C,D
A,1.000000,0.227134,0.486088,-0.577268
B,0.227134,1.000000,0.044957,0.008649
C,0.486088,0.044957,1.000000,-0.151710
D,-0.577268,0.008649,-0.151710,1.000000


 Применение функции к данным. Для примера посчитаем разброс значений — разница максимума и минимума.

In [ ]:
df.apply(lambda x: x.max() - x.min())

,0
A,4.161094
B,2.840402
C,3.715032
D,2.839896


### 2. Объединение таблиц.  Функция pd.concat
Соединение таблиц вдоль выбранной оси

`pd.concat(objs, axis=0, join='outer', ignore_index=False, copy=True, ...)`

* `objs` &mdash; объединяемые таблицы;
* `axis` : {`0` или `'index'`, `1` или `'columns'`} &mdash; ось индексов или ось колонок, иными словами соединение по вертикали или по горизонтали;
* `join` : {`'inner'`, `'outer'`} &mdash; тип объединения &mdash; пересечение или объединение индексов/колонок;
* `ignore_index` &mdash; сохранить индексы или определить и как $0, ..., n-1$;
* `copy` &mdash; копировать данные или нет.

------------

Простой пример соединения таблиц:

In [ ]:
other = df[:4].copy()  # Полное копирование
other['flag'] = other['D'] > 0
other['D'] = other['D'] ** 2
pd.concat([df, other], axis=0, join='inner')

,A,B,C,D
0,0.356587,0.677383,-0.808678,0.436242
1,0.118757,1.001112,0.271759,0.409493
2,0.341055,-0.810651,0.393015,-0.754005
3,-1.186959,0.289895,0.884809,2.085891
4,0.272135,1.395138,0.687555,0.075547
5,1.806818,-0.458174,2.906355,-0.256508
6,0.618698,-0.470176,-0.559366,0.329487
7,1.677485,2.029751,0.783604,-0.513796
8,-0.821490,0.948548,0.817842,0.378035
9,-2.354276,-0.497804,-0.779210,0.383991


In [ ]:
pd.concat([df, other], axis=1, join='inner')

,A,B,C,D,A,B,C,D,flag
0,0.356587,0.677383,-0.808678,0.436242,0.356587,0.677383,-0.808678,0.190307,True
1,0.118757,1.001112,0.271759,0.409493,0.118757,1.001112,0.271759,0.167685,True
2,0.341055,-0.810651,0.393015,-0.754005,0.341055,-0.810651,0.393015,0.568524,False
3,-1.186959,0.289895,0.884809,2.085891,-1.186959,0.289895,0.884809,4.350941,True


### 3. Группировка
Часто на практике необходимо вычислять среднее по каким-либо категориям или группам в данных. Группа может определяться, например, столбцом в таблице, у которого не так много значений. Мы хотели бы для каждого такого значения посчитать среднее значение другой колонки данных в этой группе.

Этапы группировки данных:

* разбиение данных на группы по некоторым критериям;
* применение функции отдельно к каждой группе;
* комбинирование результата в структуру данных.

Группировка выполняется функцией

`df.groupby(by=None, axis=0, level=None, sort=True, ...)`

* `df` &mdash; таблица, данные которой должны быть сгруппированы;
* `by` &mdash; задает принцип группировки. Чаще всего это имя столбца, по которому нужно сгруппировать. Может так же быть функцией;
* `axis` &mdash; ось (0 = группировать строки, 1 = группировать столбцы);
* `level` &mdash; если ось представлена мультииндексом, то указывает на уровень мультииндекса;
* `sort` &mdash; сортировка результата по индексу.

Результатом группировки является объект, состоящий из пар (имя группы, подтаблица). Имя группы соответствует значению, по которому произведена группировка. К объекту-результату группировки применимы, например, следующие операции:

* `for name, group in groupped: ... ` &mdash; цикл по группам;
* `get_group(name)` &mdash; получить таблицу, соответствующую группе с именем `name`;
* `groups` &mdash; получить все группы в виде словаря имя-подтаблица;
* `count()` &mdash; количество значений в группах, исключая пропуски;
* `size()` &mdash; размер групп;
* `sum()`, `max()`, `min()`;
* `mean()`, `median()`, `var()`, `std()`, `corr()`, `quantile(q)`;
* `describe()` &mdash; вывод описательных статистик;
* `aggregate(func)` &mdash; применение функции (или списка функций) `func` к группам.
---------

In [ ]:
df = pd.DataFrame({
    'Животное' : ['Котик', 'Песик', 'Котик', 'Песик',
                  'Котик', 'Песик', 'Котик', 'Песик'],
    'Цвет шерсти' : ['белый', 'белый', 'коричневый', 'черный',
                     'коричневый', 'коричневый', 'белый', 'черный'],
    'Рост' : sps.gamma(a=12, scale=3).rvs(size=8),
    'Длина хвостика' : sps.gamma(a=10).rvs(size=8)
})

df

,Животное,Цвет шерсти,Рост,Длина хвостика
0,Котик,белый,34.962157,10.010441
1,Песик,белый,43.403794,8.785406
2,Котик,коричневый,38.584586,14.049003
3,Песик,черный,30.754415,4.935612
4,Котик,коричневый,58.133552,13.194847
5,Песик,коричневый,18.285619,10.689200
6,Котик,белый,42.101107,6.517781
7,Песик,черный,20.315380,12.746186


Пример 1

Если все котики встанут друг на друга, то какой их суммарный рост? А у песиков? А какова суммарная длина хвостиков у котиков и у песиков?
Группировка по одной колонке и последующее применение операции суммирования:

In [ ]:
df.groupby('Животное')


In [ ]:
df.groupby('Животное').sum()

,Цвет шерсти,Рост,Длина хвостика
Животное,,,
Котик,белыйкоричневыйкоричневыйбелый,199.643850,34.620189
Песик,белыйчерныйкоричневыйчерный,130.801194,33.526261


In [ ]:
df.groupby('Животное').describe()

Рост                                                        \
         count       mean       std        min        25%        50%   
Животное                                                               
Котик      4.0  49.910963  9.484361  37.353325  45.941473  51.291197   
Песик      4.0  32.700298  6.870603  25.528567  28.060418  32.146575   

                               Длина хвостика                                \
                75%        max          count      mean       std       min   
Животное                                                                      
Котик     55.260686  59.708131            4.0  8.655047  1.221835  7.546485   
Песик     36.786456  40.979476            4.0  8.381565  1.456749  6.900436   

                                                  
               25%       50%       75%       max  
Животное                                          
Котик     7.900073  8.366687  9.121661  10.34033  
Песик     7.451240  8.188777  9.119103  10.24827

Прмер 2

Теперь предположим, что котики и песики встают только на представителей своего вида и своего цвета шерсти. Что тогда будет?

Группировка по двум колонкам и последующее применение операции суммирования

In [ ]:
df.groupby(['Животное', 'Цвет шерсти']).sum()

Рост  Длина хвостика
Животное Цвет шерсти                            
Котик    белый         86.157514       15.564421
         коричневый   113.486336       19.055768
Песик    белый         28.904368       10.248270
         коричневый    40.979476        7.634841
         черный        60.917350       15.643150

Полученная таблица имеет *мультииндекс*

In [ ]:
df.groupby(['Животное', 'Цвет шерсти']).sum().index

MultiIndex([('Котик',      'белый'),
            ('Котик', 'коричневый'),
            ('Песик',      'белый'),
            ('Песик', 'коричневый'),
            ('Песик',     'черный')],
           names=['Животное', 'Цвет шерсти'])

### 4 Функция `pd.pivot_table`

Эксель-подобные сводные таблицы

`pd.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')`

* `data` &mdash; исходная таблица;
* `values` &mdash; аггригируемый столбец, его значения непосредственно определяют значения сводной таблицы;
* `index` &mdash; ключи для группировки, относятся к индексам сводной таблицы;
* `columns` &mdash; ключи для группировки, относятся к столбцам сводной таблицы;
* `aggfunc` &mdash; функция, которая будет применена к каждой группе значений `values`, сгруппированным по значениям `index` и `columns`. Значения этой функции и есть значения сводной таблицы. Если передается список функций, то сводная таблица имеет иерархические имена колонок, верхние значения которых &mdash; имена функций;
* `fill_value` &mdash; значения для замены пропусков;
* `dropna` &mdash; не включать столбцы, которые состоят только из `NaN`;
* `margins` &mdash; добавляет результирующий столбец/строку;
* `margins_name` &mdash; имя результирующего столбец/строку.

In [ ]:
df = pd.DataFrame({
    'Животное' : ['Котик', 'Песик', 'Котик', 'Песик',
                  'Котик', 'Песик', 'Котик', 'Песик'],
    'Цвет шерсти' : ['белый', 'белый', 'коричневый', 'черный',
                     'коричневый', 'коричневый', 'белый', 'черный'],
    'Рост' : sps.gamma(a=12, scale=3).rvs(size=8),
    'Длина хвостика' : sps.gamma(a=10).rvs(size=8)
})

df

,Животное,Цвет шерсти,Рост,Длина хвостика
0,Котик,белый,32.686608,13.669965
1,Песик,белый,43.294804,4.999022
2,Котик,коричневый,18.014102,7.251468
3,Песик,черный,27.270031,10.247419
4,Котик,коричневый,38.852794,9.816088
5,Песик,коричневый,49.887211,12.353438
6,Котик,белый,35.897030,7.856139
7,Песик,черный,16.005159,12.375531


In [ ]:
pd.pivot_table(df, index='Животное', columns='Цвет шерсти', margins=True, aggfunc='sum') # default mean

Длина хвостика                                        Рост  \
Цвет шерсти          белый коричневый    черный        All       белый   
Животное                                                                 
Котик            21.526104  17.067556       NaN  38.593661   68.583638   
Песик             4.999022  12.353438  22.62295  39.975410   43.294804   
All              26.525126  29.420994  22.62295  78.569070  111.878442   

                                               
Цвет шерсти  коричневый    черный         All  
Животное                                       
Котик         56.866896       NaN  125.450534  
Песик         49.887211  43.27519  136.457205  
All          106.754107  43.27519  261.907739

## GIL (Global Interpreter Lock)

GIL (Global Interpreter Lock) — это механизм, используемый интерпретатором CPython для предотвращения одновременного выполнения нескольких потоков Python. Это глобальная блокировка, которая гарантирует, что в каждый момент времени только один поток выполняет байт-код Python, даже если программа имеет несколько потоков.

Напоминание по IO-bound и CPU-bound задачи.

#### Как обойти GIL?

1. **Использование процессов (`multiprocessing`)**:
   - Процессы создают независимые экземпляры интерпретатора Python, что позволяет выполнять задачи параллельно на разных ядрах процессора.

2. **Использование встроенных модулей на C**:
   - Некоторые модули, такие как `numpy`, освобождают GIL для выполнения своих вычислений.


Ноутбук с примитивами парралелизма
https://colab.research.google.com/drive/1XBNkerJ4RLSu3TLNH9KdBr4qoJQ-LVyO?usp=sharing#scrollTo=nM3LWp4pGaEe

## Решаем задачи

 ### Асинхронность

1. **Асинхронная фильтрация данных с использованием `yield` (0.5б)**

   Реализуйте генератор `async_filtered_numbers`, который принимает числа и фильтрует их по условию. Используйте `yield`, чтобы передать только те числа, которые соответствуют условию (например, четные).

   **Пример использования:**
   ```python
   async def async_filtered_numbers(numbers, filter_func):
       pass  # Реализуйте фильтрацию чисел с помощью yield

   async for number in async_filtered_numbers([1, 2, 3, 4, 5, 6], lambda x: x % 2 == 0):
       print(number)  # Ожидаемый вывод: 2, 4, 6
   ```


2. **Асинхронное вычисление факториалов с ограничением по времени (1б)**

   Создайте корутину `async_factorial`, которая принимает целое число и возвращает его факториал. Используйте цикл для выполнения вычислений. На каждой итерации цикла добавьте небольшую задержку, чтобы дать другим задачам возможность выполняться, с помощью `await asyncio.sleep(0)`. Если выполнение займет больше 0.1 секунды, корутина должна прерывать вычисления и выбросить исключение `asyncio.TimeoutError`. Необходимо, чтобы сначала было напечатано успешно вычисленное значение (все значения), и только потом выброшено исключение.

   **Пример использования:**
   ```python
   async def async_factorial(n):
       # Реализуйте вычисление факториала с ограничением времени. Можно начать так...
       result = 1
       start_time = asyncio.get_event_loop().time()  # Текущее время

       for i in range(1, n + 1):
            result *= i
            # Добавим небольшую задержку, чтобы дать другим задачам возможность выполняться
            await asyncio.sleep(0)

            # Ваш код
   
   # Реорганизуйте код ниже, чтобы было выполнено последнее условие
   print(await async_factorial(10000))
   print(await async_factorial(5))
   ```

   3. **Асинхронная обработка задач с управлением через `asyncio.wait` (1б)**

   Напишите корутину `task_handler`, которая принимает несколько задач и управляет их выполнением, используя `asyncio.wait`. Обратите внимание, что возвращает `asyncio.wait`. Пусть задачи завершаются только, когда будут выполнены три самые короткие по времени задачи. Остальные задачи должны остаться в состоянии ожидания.

   **Пример использования:**
   ```python
   async def sample_task(name, delay):
       await asyncio.sleep(delay)
       print(f"{name} завершена после {delay} секунд")

   await task_handler([
       sample_task("Task1", 5),
       sample_task("Task2", 3),
       sample_task("Task3", 1),
       sample_task("Task4", 4),
       sample_task("Task5", 2)
   ])
   # Ожидаемый вывод: Выполнение трех задач с наименьшим временем, остальные задачи не завершаются
   ```

   4. **Асинхронная очередь с приоритетом и таймаутом (2б)**

   Создайте асинхронную очередь с приоритетом. Пусть `producer` добавляет в очередь задачи с разным приоритетом, а `consumer` обрабатывает задачи в порядке приоритета. При этом задачи, которые находятся в очереди более 3 секунд, должны удаляться без обработки. Если очередь пуста в течение времени ожидания в 1 секунду, завершаем с сообщением "No tasks left to process." Это должно происходить в конце (потому что priority_producer не будет писать в конце). Убедитесь, что это сработает и в середине, использовав закомментированное `# await asyncio.sleep(3)`. Считайте, что `priority_consumer` требует 1.5 секунды на обработку одной задачи (вставьте `await asyncio.sleep(1.5)` после операции получения очередного запроса из очереди с учетом таймаута).

   **Пример использования:**
   ```python
   async def priority_producer(queue):
       # Добавляем задачи с приоритетами и временной меткой (чем меньше приоритет, тем выше он в очереди)
       await queue.put((1, time.time(), "High Priority Task"))
       await asyncio.sleep(1)
       await queue.put((3, time.time(), "Low Priority Task"))
       await asyncio.sleep(1) # await asyncio.sleep(3)
       await queue.put((2, time.time(), "Medium Priority Task"))
       await asyncio.sleep(1)
       await queue.put((0, time.time(), "Very High Priority Task"))

   async def priority_consumer(queue):
       pass  # Извлекайте задачи из очереди в порядке приоритета с учетом таймаута. Не забудьте отмечать выполненные задачи через queue.task_done()

   queue = asyncio.PriorityQueue()
   await asyncio.gather(priority_producer(queue), priority_consumer(queue))
   ```

   Ожидаемый вывод
   ```
   Processing High Priority Task with priority 1
   Processing Low Priority Task with priority 3
   Processing Medium Priority Task with priority 2
   Task 'Very High Priority Task' was discarded due to timeout.
   No tasks left to process.
   ```

   5. **Асинхронный объединитель данных из нескольких источников с фильтрацией (0.5б)**

   Реализуйте `merge_data_sources`, который объединяет данные из нескольких асинхронных источников. Используйте `yield from`, чтобы делегировать обработку данных. Добавьте фильтр, который будет удалять пустые данные.

   **Пример использования:**
   ```python
   async def data_source_one():
      for data in ["Data1", None, "Data3", ""]:
        await asyncio.sleep(1)
        yield data

   async def data_source_two():
      for data in ["Valid1", "", "Valid3"]:
        await asyncio.sleep(1)
        yield data

   async def merge_data_sources():
       pass  # Объедините данные из нескольких источников с фильтрацией и обработкой ошибок

   async for data in merge_data_sources(data_source_one, data_source_two):
       print(data)  # Ожидаемый вывод: Только непустые и валидные строки
   ```

   6. **Асинхронный агрегатор данных из API с несколькими типами запросов (3б)**

   Реализуйте корутину `api_aggregator`, которая отправляет асинхронные запросы к нескольким API. Пусть будут три типа запросов:
   - **Запросы типа "fast"** обрабатываются без задержки,
   - **Запросы типа "slow"** обрабатываются с задержкой 3 секунды,
   - **Запросы типа "unstable"** могут завершиться ошибкой с вероятностью 50%.

   `api_aggregator` должен запускать все запросы параллельно и обрабатывать результаты по мере их поступления. При ошибке запросов "unstable" должен происходить повторный запрос до трех раз. Запустите своё решение несколько раз и убедитесь, что все сценарии корректно отрабатывают

   **Пример использования:**
   ```python
   async def fast_request():
       return "Fast result"

   async def slow_request():
       await asyncio.sleep(3)
       return "Slow result"

   async def unstable_request():
       if random.random() > 0.5:
           raise ValueError("Unstable request failed")
       return "Unstable result"

   async def api_aggregator():
       pass  # Реализуйте агрегатор для запуска всех запросов и обработки результатов

   await api_aggregator()
   ```


## Решение Асинхронные задачи

In [ ]:
import nest_asyncio
nest_asyncio.apply()

async def async_filtered_numbers(numbers, filter_func):
     for number in numbers:
         if filter_func(number):
             yield number

async for number in async_filtered_numbers([1, 2, 3, 4, 5, 6], lambda x: x % 2 == 0):
    print(number)

2
4
6


In [ ]:
import asyncio

async def async_factorial(n):
    # Реализуйте вычисление факториала с ограничением времени. Можно начать так...
    result = 1
    start_time = asyncio.get_event_loop().time()  # Текущее время

    for i in range(1, n + 1):
         result *= i
         # Добавим небольшую задержку, чтобы дать другим задачам возможность выполняться
         await asyncio.sleep(0)
         if asyncio.get_event_loop().time() - start_time > 0.1:
            raise asyncio.TimeoutError("Время вычисления превышено")
    return result


# Реорганизуйте код ниже, чтобы было выполнено последнее условие
print(await async_factorial(5))
print(await async_factorial(10000))

120
...


In [ ]:
async def sample_task(name, delay):
    await asyncio.sleep(delay)
    print(f"{name} завершена после {delay} секунд")

async def task_handler(tasks):
    wrapped_tasks = [asyncio.create_task(task) for task in tasks]

    completed_tasks = []
    while len(completed_tasks) < 3:
        done, pending = await asyncio.wait(wrapped_tasks, return_when=asyncio.FIRST_COMPLETED)
        for task in done:
            complet5ed_tasks.append(task)
            wrapped_tasks.remove(task)

    for task in done:
        try:
            await task
        except asyncio.CancelledError:
            pass

    for task in pending:
        task.cancel()

await task_handler([
    sample_task("Task1", 5),
    sample_task("Task2", 3),
    sample_task("Task3", 1),
    sample_task("Task4", 4),
    sample_task("Task5", 2)
])


# Ожидаемый вывод: Выполнение трех задач с наименьшим временем, остальные задачи не завершаются

Task3 завершена после 1 секунд
Task5 завершена после 2 секунд
Task2 завершена после 3 секунд


In [ ]:
import asyncio
import time

async def priority_producer(queue):
    # Добавляем задачи с приоритетами и временной меткой (чем меньше приоритет, тем выше он в очереди)
    await queue.put((1, time.time(), "High Priority Task"))
    await asyncio.sleep(1)
    await queue.put((3, time.time(), "Low Priority Task"))
    await asyncio.sleep(1)
    await queue.put((0, time.time(), "Medium Priority Task"))
    await asyncio.sleep(1)
    await queue.put((2, time.time(), "Very High Priority Task"))

async def priority_consumer(queue):
    while True:
        try:
            # Ожидаем задачи с таймаутом на извлечение
            priority, timestamp, task = await asyncio.wait_for(queue.get(), timeout=1)
            await asyncio.sleep(1.5)

            # Проверяем, не истекло ли время ожидания задачи
            if time.time() - timestamp > 3:
                print(f"Task '{task}' was discarded due to timeout.")
            else:
                print(f"Processing {task} with priority {priority}")

            queue.task_done()  # Отмечаем задачу как выполненную

        except asyncio.TimeoutError:
            # Если очередь пуста в течение времени ожидания, завершаем
            if queue.empty():
                print("No tasks left to process.")
                break

async def main():
    queue = asyncio.PriorityQueue()
    await asyncio.gather(priority_producer(queue), priority_consumer(queue))

# Запуск корутин main
asyncio.run(main())


Processing High Priority Task with priority 1
Processing Low Priority Task with priority 3
Processing Medium Priority Task with priority 0
Task 'Very High Priority Task' was discarded due to timeout.
No tasks left to process.


In [ ]:
async def data_source_one():
   for data in ["Data1", None, "Data3", ""]:
     await asyncio.sleep(1)
     yield data

async def data_source_two():
   for data in ["Valid1", "", "Valid3"]:
     await asyncio.sleep(1)
     yield data

async def merge_data_sources(*sources):
    for source in sources:
        async for data in source():
            if data:
                yield data

async for data in merge_data_sources(data_source_one, data_source_two):
    print(data)  # Ожидаемый вывод: Только непустые и валидные строки

Data1
Data3
Valid1
Valid3


In [ ]:
import random

async def fast_request():
    return "Fast result"

async def slow_request():
    await asyncio.sleep(3)
    return "Slow result"

async def unstable_request():
    if random.random() > 0.25:
        raise ValueError("Unstable request failed")
    return "Unstable result"

async def retry_unstable_request(attempts=3):
    for attempt in range(attempts):
        try:
            result = await unstable_request()
            return result
        except ValueError as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == attempts - 1:
                return "Unstable request failed after 3 attempts"


async def api_aggregator():
    tasks = [
        asyncio.create_task(fast_request()),
        asyncio.create_task(slow_request()),
        asyncio.create_task(retry_unstable_request()),
    ]

    for task in asyncio.as_completed(tasks):
        try:
            result = await task
            print(result)
        except Exception as e:
            print(f"Error occurred: {e}")


await api_aggregator()

Attempt 1 failed: Unstable request failed
Attempt 2 failed: Unstable request failed
Attempt 3 failed: Unstable request failed
Fast result
Unstable request failed after 3 attempts
Slow result
